# Homework 3

# Finite difference methods.

## Explicit Finite Difference

Our goal is to value European options with $V(S, t)$.

### Solving the PDE

We will follow the Black-Scholes assumption that the underlying stock follows this stochastic process:

$$dS_t = rS_t dt + \sigma S_t dW_t$$

Then the price of the European option must satisfy this PDE:

$$\frac{\partial V}{\partial t} + rS \frac{\partial V}{\partial S} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} - rV = 0$$

In order to solve this PDE, we must have constant coefficients. We can do this through a change of variable, modeling returns instead of the actual stock price.

$$S = e^x$$
$$x = \ln S$$

Then we will get a new value equation $u$ where
$$V(S, t) = V(e^x, t) = u(x, t)$$
$$\frac{\partial V}{\partial t}(t, S) = \frac{\partial u}{\partial t}(t, x)$$

And using Ito's lemma, our original Black-Scholes PDE becomes

$$\frac{\partial u}{\partial t} + \nu\frac{\partial u}{\partial x} + \frac{1}{2}\sigma^2 \frac{\partial^2 u}{\partial x^2} - ru = 0$$

where

$$\nu = r - \frac{\sigma^2}{2}$$

Merton (1973) showed that this PDE, like the heat equation, can be solved analytically and used to value options through the famous Black-Scholes equation. But we can also solve it via a numerical method, the **explicit finite difference** method.

### Discretizing the Domain

This process involves discretizing this equation, and solving it backwards from the payoff at maturity $T$.

This begins by discretizing our domain.

Our domain is $t \in [0, T]$ and $x \in (-\infty, \infty)$.

We will discretize $t$ into $n + 1$ points like so:
$$\Delta t = \frac{T}{n}$$
$$t = \{0, \Delta t, 2 \Delta t, \ldots, n \Delta t\}$$

For $x$, we must set some large boundary instead of using $\infty$, which we will define as $N$. Therefore we will have $2N + 1$ points like so:
$$x = \{-N \Delta x, (-N + 1) \Delta x, \ldots, 0, \Delta x, \ldots, N \Delta x\}$$

The value of $\Delta x$ is technically arbitrary. However, in order for this process to converge, $\Delta x$ must follow

$$\Delta x \geq \sigma \sqrt{3 \Delta t}$$

The time complexity of the explicit algorithm is $O(\Delta x^2 + \Delta t)$. Since we want to minimize the time complexity, the best choice of $\Delta x$ is in practice always $\sigma \sqrt{3\Delta t}$.

### Discretizing the Derivatives

For the explicit finite difference method, there are four points we will need. These are
- $u_{i+1, j+1}$
- $u_{i+1, j}$
- $u_{i+1, j-1}$
- $u_{i, j}$

And there are three derivatives we are trying to calculate
$$\frac{\partial u}{\partial t}, \frac{\partial u}{\partial x}, \frac{\partial^2 u}{\partial x^2}$$

We can use the limit equation for derivatives to describe finite difference for the first-order derivatives.

$$u'(x) = \lim_{h \rightarrow 0}  \frac{u(x + h) - u(x)}{h}$$

And we can also use Taylor expansion to get the limit equation for second-order derivatives in terms of the first-order equation.

$$u''(x) = \lim_{h \rightarrow 0} \frac{u(x+h) - 2u(x) + u(x-h)}{h^2}$$

The derivative with respect to $t$ most neatly fits into this paradigm. If we define
$$h = \Delta t$$
then we get
$$\frac{\partial u}{\partial t} = \frac{u_{i+1, j} - u_{i, j}}{\Delta t}$$

For the first-order derivative with respect to $x$, because we are calculating these values with respect to $u_{i, j}$, we don't want to bias it up or down, so we will use the above and below point and then average them.
$$h = \Delta x$$
$$\frac{\partial u}{\partial x} = \frac{u_{i+1,j+1} - u_{i+1,j-1}}{2\Delta x}$$

And for the second-order derivative with respect to $x$ we will use the corresponding limit equation.

$$h = \Delta x$$
$$\frac{\partial^2 u}{\partial x^2} = \frac{u_{i+1,j+1} - 2u_{i+1,j} + u_{i+1,j-1}}{\Delta x^2}$$

### The Discretized Equation

$$\frac{\partial u}{\partial t} + \nu\frac{\partial u}{\partial x} + \frac{1}{2}\sigma^2 \frac{\partial^2 u}{\partial x^2} - ru = 0$$

Substituting back these finite differences into our original equation, we get

$$\frac{u_{i+1, j} - u_{i, j}}{\Delta t} + \nu \frac{u_{i+1,j+1} - u_{i+1,j-1}}{2\Delta x} + \frac{1}{2} \sigma^2 \frac{u_{i+1,j+1} - 2u_{i+1,j} + u_{i+1,j-1}}{\Delta x^2} - r u_{i+1, j} = 0$$

Expand out the equation.
$$\frac{u_{i+1, j}}{\Delta t} - \frac{u_{i, j}}{\Delta t} + \frac{\nu}{2\Delta x} u_{i+1, j+1} - \frac{\nu}{2\Delta x} u_{i+1, j-1} + \frac{\sigma^2}{2\Delta x^2} u_{i+1, j+1} - \frac{\sigma^2}{\Delta x^2} u_{i+1, j} + \frac{\sigma^2}{2 \Delta x^2} h - ru_{i+1, j} = 0$$

Rearrange to solve for $u_{i, j}$.
$$u_{i, j} = \Delta t \left( \frac{\sigma^2}{2\Delta x^2} u_{i+1, j+1} + \frac{\nu}{2\Delta x} u_{i+1, j+1} \right) + \Delta t \left( \frac{\sigma^2}{2\Delta x^2} u_{i+1, j-1} - \frac{\nu}{2\Delta x} u_{i+1, j-1} \right) + u_{i+1, j} - \Delta t \frac{\sigma^2}{\Delta x^2} u_{i+1, j} - ru_{i+1, j} \Delta t$$

Factor out the probabilities.
$$u_{i, j} = p_u u_{i+1, j+1} + p_m u_{i+1, j} + p_d u_{i+1, j-1}$$

$$p_u = \Delta t \left( \frac{\sigma^2}{2\Delta x^2} + \frac{\nu}{2\Delta x} \right)$$
$$p_m = 1 - \Delta t \frac{\sigma^2}{\Delta x^2} - r \Delta t$$
$$p_d = \Delta t \left( \frac{\sigma^2}{2\Delta x^2} - \frac{\nu}{2\Delta x} \right)$$

### Valuation

To carry out the valuation, we will follow these steps.

1. Initialize the following constants:
    - $K$
    - $T$
    - $S$
    - $\sigma$
    - $r$
    - $\delta$
    - $n$
    - $N$
    - $dt$
    - $dx$
    - $\nu$
    - $p_u$
    - $p_m$
    - $p_d$
2. Create a vector of asset prices at maturity.
3. Initialize option values at maturity based on the option payoff formula.
4. Step backwards through the lattice by solving the discretized equation for each point in each time step based on the three points in the next time step.
5. For the boundary conditions, initialize them based on the option type.
6. Return the value at (0, 0)

## Implicit Finite Difference

Use the same discretized equation for explicit finite difference, but calculate the derivative with respect to $x$ at time step $i$ not $i + 1$.

$$\frac{u_{i+1, j} - u_{i, j}}{\Delta t} + \nu \frac{u_{i,j+1} - u_{i,j-1}}{2\Delta x} + \frac{1}{2} \sigma^2 \frac{u_{i,j+1} - 2u_{i,j} + u_{i,j-1}}{\Delta x^2} - r u_{i, j} = 0$$

We can perform the same steps of expansion and factoring.

Expand out the equation.
$$\frac{u_{i+1, j}}{\Delta t} - \frac{u_{i, j}}{\Delta t} + \frac{\nu}{2\Delta x} u_{i, j+1} - \frac{\nu}{2\Delta x} u_{i, j-1} + \frac{\sigma^2}{2\Delta x^2} u_{i, j+1} - \frac{\sigma^2}{\Delta x^2} u_{i, j} + \frac{\sigma^2}{2 \Delta x^2} h - ru_{i, j} = 0$$

Rearrange to solve for $u_{i, j}$.
$$u_{i + 1, j} = \Delta t \left( -\frac{\sigma^2}{2\Delta x^2} u_{i, j+1} - \frac{\nu}{2\Delta x} u_{i, j+1} \right) + \Delta t \left( -\frac{\sigma^2}{2\Delta x^2} u_{i, j-1} + \frac{\nu}{2\Delta x} u_{i, j-1} \right) + u_{i, j} + \Delta t \frac{\sigma^2}{\Delta x^2} u_{i, j} - ru_{i, j} \Delta t$$

Factor out the constants (not probabilities!).
$$u_{i + 1, j} = A u_{i, j+1} + B u_{i, j} + C u_{i, j-1}$$

$$A = -\frac{1}{2} \Delta t \left( \frac{\sigma^2}{\Delta x^2} + \frac{\nu}{\Delta x} \right)$$
$$B = 1 + \Delta t \frac{\sigma^2}{\Delta x^2} + r \Delta t$$
$$C = -\frac{1}{2} \Delta t \left( \frac{\sigma^2}{\Delta x^2} - \frac{\nu}{\Delta x} \right)$$

We can construct a system of equations that represents the time step from $t_i$ to $t_{i + 1}$. For each column $V$ which has $2N + 1$ rows,

$$A V_{i, N - 1} + B V_{i, N - 1} + C V_{i, N - 1} = V_{i, N - 1}$$
$$A V_{i, N - 2} + B V_{i, N - 2} + C V_{i, N - 2} = V_{i, N - 2}$$
...
$$A V_{i, -N + 1} + B V_{i, -N + 1} + C V_{i, -N + 1} = V_{i, -N + 1}$$

Note here that because the value of each point depends on the past above and below points, these equations do not work for the boundaries at the top and bottom.

For these, we set the boundary condition based on the delta of the option as the stock price goes to infinity/zero.
The finite difference of the delta here is calculated at the top as
$$V_{i, N} - V_{i, N - 1} = \lambda_U$$
and at the bottom as
$$V_{i, -N + 1} - V_{i, -N} = \lambda_L$$


The value of $\lambda_U$/$\lambda_L$ depends on the option type.

For a call option, as the stock price approaches infinity, the payoff from the option just keeps adding on the stock price, giving them a linear relationship with a delta of 1.
$$\lambda_{Uc} = S \uparrow \infty: \frac{\partial V_c}{\partial S} = 1$$
And as the stock price approaches zero, the call will never be exercised and the payoff doesn't change, giving a delta of 0.
$$\lambda_{Lc} = S \downarrow 0: \frac{\partial V_c}{\partial S} = 0$$

For a put option, as the stock price approaches infinity, the put will never be exercised giving a delta of 0.
$$\lambda_{Up} = S \uparrow \infty: \frac{\partial V_p}{\partial S} = 0$$

And as the stock price approaches 0, the put payoff will have the same linear relationship with the call, but it increases when the price decreases, making the delta -1.

$$\lambda_{Dp} = S \downarrow 0: \frac{\partial V_p}{\partial S} = -1$$



Putting all of this together, we can construct a matrix equation of form $Ax = b$ to represent this systems of equations.

$$\begin{bmatrix} 1 & - 1 & 0 & 0 & 0 & \ldots & 0 \\ A & B & C & 0 & 0 & \ldots & 0 \\ 0 & A & B & C & 0 & \ldots & 0 \\ \vdots & \ddots & \ddots & \ddots & \ddots & \ddots & \vdots \\ 0 & 0 & \ddots & \ddots & B & C & 0 \\ 0 & 0 & 0 & \ddots & A & B & C \\ 0 & 0 & 0 & \ldots & 0 & 1 & -1 \end{bmatrix} \begin{bmatrix} V_{i, N} \\ V_{i, N - 1} \\ V_{i, N - 2} \\ \vdots \\ \vdots \\ V_{i, -N + 1} \\ V_{i, -N} \end{bmatrix} = \begin{bmatrix} \lambda_U \\ V_{i+1,N-1} \\ V_{i+1, N-2} \\ \vdots \\ \vdots \\ V_{i+1,-N+1} \\ \lambda_L \end{bmatrix}$$

Normally, this calculation would be done by inverting the matrix. However, for sufficiently large $N$, this becomes computationally infeasible. Fortunately, there is a recurrence procedure for calculating tridiagonal matrices of these forms in $O(n)$.

The procedure is derived in the following way.

For simplicity of notation, I will describe $V_{i, j}$ by $x_j$ and $V_{i, + 1, j}$ as $y_j$ and index from 1 to $n$.

The first equation is
$$x_1 - x_2 = y_1$$
which we can rearrange to solve for $x_1$ in terms of $x_2$
$$x_1 = y_1 + x_2$$
I can perform a change of variable here where I define
$$D_1 = y_1$$
$$E_1 = 1.0$$
so that
$$x_1 = D_1 + E_1 x_2$$
This is the initial condition for the recurrence, as we will see.

The second equation is
$$Ax_1 + Bx_2 + Cx_3 = y_2$$
We can substitute in the first equation like so
$$A(y_1 + x_2) + Bx_2 + Cx_3 = y_2$$
and rearrange to solve for $x_2$ in terms of $x_3$
$$Ay_1 + Ax_2 + Bx_2 + Cx_3 = y_2$$
$$x_2 = \frac{y_2 - Ay_1}{A + B} - \frac{C}{A + B}x_3$$
Doing the same of change of variable, we can define
$$D_2 = \frac{y_2 - AD_1}{AE_1 + B}$$
$$E_2 = -\frac{C}{AE_1 + B}$$
so that
$$x_2 = D_2 + E_2 x_3$$
We can see the same structure emerge here. One more example will fully illuminate it.

The third equation is
$$Ax_2 + Bx_3 + Cx_4 = y_3$$
Substituting in the second equation,
$$A(D_2 + E_2 x_3) + Bx_3 + Cx_4 = y_3$$
Then rearranging to solve for $x_3$ in terms of $x_4$
$$AD_2 + AE_2 x_3 + Bx_3 + Cx_4 = y_3$$
$$x_3 = \frac{y_3 - AD_2}{AE_2 + B} - \frac{C}{AE_2 + B}x_4$$
With the change of variable,
$$D_3 = \frac{y_3 - AD_2}{AE_2 + B}$$
$$E_3 = -\frac{C}{AE_2 + B}$$

The recurrence now becomes clear. Each equation follows this form.
$$x_j = D_j + E_j x_{j+1}$$
where
$$D_j = \frac{y_j - AD_{j - 1}}{AE_{j - 1} + B}$$
$$E_j = -\frac{C}{AE_{j - 1} + B}$$

We have our initial condition, our recurrence, but where's our stopping point? The second-to-last equation, expressed in terms of $D$ and $E$, is
$$x_{n-1} = D_{n-1} + E_{n-1} x_n$$

And the last equation is
$$x_{n-1} - x_n = y_n$$
Substituting the previous equation into this one, we get
$$D_{n-1} + E_{n-1} x_n - x_n = y_n$$
Then rearranging to solve for $x_n$
$$x_n = \frac{y_n - D_{n-1}}{E_{n-1} - 1.0}$$
which is a complete solution.

The tridiagonal system solver procedure then follows like so:

1. Create vectors for $D$ and $E$ of size $n$ and set their first element to $y_1$ and $1.0$, respectively.
2. Forward-substitute the values for $D$ and $E$ using the $D$ and $E$ recurrences.
3. Create a vector for $x$ and solve for $x_n$ directly.
4. Back-substitute each element in $x$ through the $x$ recurrence.

This solver is run at each time step in the same way that explicit finite difference is, and then the final option price is the value of $V_{0, 0}$.

# Choosing Parameter Values

## Explicit Finite Difference

For explicit finite difference, as discussed previously, the best value for $\Delta x$ is always

$$\Delta x = \sigma \sqrt{3 \Delta t}$$

Our order of convergence is $O(\Delta x^2 + \Delta t)$, so our goal of converging to $\epsilon$ is satisfied by
$$\Delta x^2 + \Delta t = \epsilon$$
which we can substitute and rearrange to find the correct value for $\Delta t$
$$\sigma^2 3 \Delta t + \Delta t = \epsilon$$
$$\Delta t = \frac{\epsilon}{1 + 3 \sigma^2}$$

The value for $n$ naturally falls out of this via $n = \frac{T}{\Delta t}$ as for all these procedures. And we can define $N = n$ (although I can't find a reason to do this beyond "the nature of the explicit scheme", either in Mariani & Florescu (2019) or Clewlow (1998)).

## Implicit Finite Difference

The order of convergence is the same as with explicit finite difference, but the method is unconditionally stable and convergent. We can arbitrarily choose for the error to be divided equally between $\Delta t$ and $\Delta x$.

$$\Delta t = \frac{\epsilon}{2}$$
$$\Delta x = \sqrt{\frac{\epsilon}{2}}$$

$n$ is computed as before, and we choose $N = n$ to be the same order of magnitude as $n$ as suggested by Mariani & Florescu (2019). (M & F cites Clewlow (1998) for this but I can't find any source for that in there.)

# Calculation

- Explicit Method - European Call Price: 9.72838461521403
    - $n$: 11200
    - $N$: 11200

- Explicit Method - European Put Price: 5.884952006728757
    - $n$: 11200
    - $N$: 11200

- Implicit Method - European Call Price: 9.72734157812497
  - $n$: 20000
  - $N$: 20000

- Implicit Method - European Put Price: 5.883910596683879
  - $n$: 20000
  - $N$: 20000

# Empirical Convergence

There was some error in my implicit finite difference code that caused it to fail to converge.